In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

data = pd.read_csv('processed_data/with_ranges_features.csv')

# Determine the number of classes and create a global label encoder


In [27]:
# Extract features and label
df = data[['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient', 'HeartRateClass']]
df.dropna()

scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)
    
features = df[['Elevation', 'Distance', 'HeartRateQuotient', 'HeartRateRange', 'Cadence', 'Speed', 'SpeedRange', 'SpeedQuotient']].to_numpy()
features = features.reshape(features.shape[0], 1, features.shape[1])

labels = df['HeartRateClass'].to_numpy()
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['HeartRateClass'])

num_classes = len(label_encoder.classes_)

labels = to_categorical(labels, num_classes=num_classes)
    
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [28]:
from tcn import TCN, compiled_tcn

def build_model(middle_layer):
    model = keras.Sequential([
        keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
        middle_layer,
        keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.summary()
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=["accuracy"], loss="categorical_crossentropy")
    return model

In [29]:
def get_tcn_layer():
    return TCN(
        nb_filters=30,
        kernel_size=3,
        dilations=[2**i for i in range(6)],
    )
tcn_model = build_model(get_tcn_layer())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_2 (TCN)                 (None, 30)                31050     
                                                                 
 dense_4 (Dense)             (None, 10)                310       
                                                                 
Total params: 31,360
Trainable params: 31,360
Non-trainable params: 0
_________________________________________________________________


In [30]:
def get_lstm_layer():
    return keras.layers.LSTM(100)

lstm_model = build_model(get_lstm_layer())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100)               43600     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 44,610
Trainable params: 44,610
Non-trainable params: 0
_________________________________________________________________


2024-06-22 12:58:33.946440: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:58:33.947578: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:58:33.948382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [31]:
def fit_model(model):
    es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5, verbose=1)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f'./logs/run', histogram_freq=1)
    modelckpt_callback = keras.callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=f'model_checkpoint.weights.h5',
        verbose=1,
        save_weights_only=True,
        save_best_only=True,
    )

    # Train the model
    return model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=3,
        callbacks=[es_callback, tensorboard_callback, modelckpt_callback]
    )

In [32]:
lstm_history = fit_model(lstm_model)

Epoch 1/3


2024-06-22 12:58:45.206377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:58:45.207289: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:58:45.208175: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

41380/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 3.7760e-06

2024-06-22 12:59:43.320486: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-22 12:59:43.321766: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-22 12:59:43.322569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus


Epoch 1: val_loss did not improve from inf
41389/41389 [==============================] - 65s 2ms/step - loss: nan - accuracy: 3.7752e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
41382/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 2.2655e-06
Epoch 2: val_loss did not improve from inf
41389/41389 [==============================] - 64s 2ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
41363/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 2.2665e-06
Epoch 3: val_loss did not improve from inf
41389/41389 [==============================] - 64s 2ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00


In [33]:
tcn_history = fit_model(tcn_model)

Epoch 1/3
41379/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 3.0209e-06
Epoch 1: val_loss did not improve from inf
41389/41389 [==============================] - 192s 5ms/step - loss: nan - accuracy: 3.0201e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
41382/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 2.2655e-06
Epoch 2: val_loss did not improve from inf
41389/41389 [==============================] - 193s 5ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
41381/41389 [============================>.] - ETA: 0s - loss: nan - accuracy: 2.2655e-06
Epoch 3: val_loss did not improve from inf
41389/41389 [==============================] - 195s 5ms/step - loss: nan - accuracy: 2.2651e-06 - val_loss: nan - val_accuracy: 0.0000e+00
